# Literature Search

Here we will explore the prevalence of different measures of aperiodic activity in the literature.

To do so, we will use [LISC](https://github.com/lisc-tools/lisc), a Python module for collecting and analyzing scientific literature. More information on LISC can be found in the [documentation](https://lisc-tools.github.io/lisc/).

To explore the prevalence and use cases of aperiodic measures, we will collect information from the Pubmed database on the number of papers that include search terms mentioning aperiodic measures, as well as the number of papers that mention these terms of interest in co-occurrence with specified association terms.

Note that collecting literature data in this way is likely an under-estimate of the use of these measures, as the approach employed here only searches for search terms occurring in titles and abstracts.

In [ ]:
# Setup notebook state
from nbutils import setup_notebook; setup_notebook()

In [ ]:
import seaborn as sns

In [ ]:
# Import LISC code for literature collection & analysis
from lisc import Counts, Counts1D
from lisc.collect import collect_info, collect_across_time
from lisc.io import (save_meta_data, load_meta_data, save_object, load_object,
                     save_time_results, load_time_results)
from lisc.plts.counts import plot_matrix, plot_vector
from lisc.plts.time import plot_results_across_years

In [ ]:
# Import custom project code
from apm.io import APMDB
from apm.plts.utils import figsaver

## Settings

First, we will define some settings for the literature collection.

In [ ]:
# Settings for literature collection
db = 'pubmed'
#db = 'pmc'

In [ ]:
# Notebook settings
RUN_COLLECTION = False
SAVE_DATA = False

In [ ]:
# File path settings
APDB = APMDB()

In [ ]:
# Settings for saving figures
SAVE_FIG = False
FIGPATH = APDB.figs_path / '02_literature'

# Create helper function to manage figsaver settings
fsaver = figsaver(SAVE_FIG, FIGPATH, directory=True)

## Define Terms

Here we will define search terms to use to collect information on the literature of interest. 

### Search Terms

First, we will define the search terms that reflect our terms of interest.

Note that each set of terms within a list is a defined as a set of synonyms

In [ ]:
# Set list of aperiodic terms & labels
concept_terms = [
    ['aperiodic', 'arrythmic'],
    ['1/f', '1/f-like', 'one-over f'],
    ['scale free', 'fractal'],
    ['criticality', 'complex dynamics'],
    ['chaos', 'dynamical system'],
    ['neural noise'],
]
concept_labels = ['aperiodic', '1/f', 'scale free', 'criticality', 'chaos', 'neural noise']
assert len(concept_terms) == len(concept_labels)

In [ ]:
# Set list of aperiodic method categories & labels
method_terms = [
    ['autocorrelation'],
    ['dfa', 'detrended fluctuation analysis', 'hurst exponent'],
    ['fractal', 'correlation dimension', 'higuchi', 'katz', 'petrosian'],
    ['complexity'],
    ['entropy', 'information theory'],
    ['spectral parameterization', 'fooof', 'irasa', 'cgsa', 'aperiodic exponent',
     'spectral slope', 'spectral exponent', 'powerlaw decay'],
]
method_labels = ['autocorrelation', 'fluctuation', 'fractal', 'complexity', 'entropy', 'exponent']
assert len(method_terms) == len(method_labels)

In [ ]:
# Set list of specific measures
measure_terms = [
    # Autocorrelation measures
    ['autocorrelation'],
    ['autocorrelation decay', 'decorrelation time', 'autocorrelation timescale',
     'autocorrelation window', 'autocorrelation width'],
    # Fluctuation measures
    ['hurst exponent'],
    ['detrended fluctuation analysis', 'dfa', 'scaling exponent'],
    ['multiscale dfa', 'multiscale detrended fluctuation analysis', 
     'multifractal dfa', 'multifractal detrended fluctuation analysis', 
     'mfdfa', 'multiscale multifractal analysis'],
    # Fractal dimension measures
    ['correlation dimension'],
    ['higuchi fractal dimension', 'higuchi dimension'],
    ['katz fractal dimension', 'katz dimension'],
    ['petrosian fractal dimension', 'petrosian dimension'],
    # Complexity measures
    ['hjorth complexity', 'hjorth'],
    ['lempel-ziv complexity', 'LZ complexity'],
    ['lyapunov exponent', 'lyapunov exponents'],
    # Entropy measures
    ['approximate entropy'],
    ['sample entropy'],
    ['permutation entropy'],
    ['weighted permutation entropy'],
    ['spectral entropy'],
    ['multiscale entropy'],
    # Aperiodic exponent measures
    ['irasa', 'irregular resampling auto-spectral analysis', 'fractal component'],
    ['specparam', 'spectral parameterization', 'fooof', 'fitting oscillations and one-over f',
     'spectral slope', 'spectral exponent', 'powerlaw decay'],
]

### Inclusion / Exclusion Terms

In some cases, we might want to use additional search terms to explicit include or exclude results that contain particular terms. 

These terms can be used to restrict searches to literature of interest, excluding irrelevant literature. 

In [ ]:
# Set inclusion / exclusion terms
inclusions = [
    'EEG', 'electroencephalography',
    'MEG', 'magnetoencephalography',
    'ECoG', 'electrocorticography',
    'iEEG', 'intracranial EEG',
    'LFP', 'local field potential',
]

exclusions = ['lithium']

In [ ]:
# Define a list of association terms
assoc_terms = [
    ['aging', 'age-related', 'development'],
    ['sleep', 'anesthesia', 'consciousness', 'brain states', 'coma'],
    ['cognitive', 'cognition', 'task', 'behavior', 'experiment', 'perception', 'memory'],
    ['seizure', 'epilepsy', 'ictal', 'interictal'],
    ['clinical', 'disorder', 'psychopathology', 'psychiatry', 'diagnose',
     'mental illness', 'biomarker', 'pathophysiology', 'disease'],
]

### Collect terms

In [ ]:
# Collect all terms together
concept_terms_dict = {
    'terms' : concept_terms,
    'labels' : concept_labels,
    'inclusions' : [inclusions] * len(concept_terms),
    'exclusions' : [exclusions] * len(concept_terms),
}

method_terms_dict = {
    'terms' : method_terms,
    'labels' : method_labels,
    'inclusions' : [inclusions] * len(method_terms),
    'exclusions' : [exclusions] * len(method_terms),
}

measure_terms_dict = {
    'terms' : measure_terms,
    'inclusions' : [inclusions] * len(measure_terms),
    'exclusions' : [exclusions] * len(measure_terms),
}

## Run a Metadata Check

In [ ]:
if RUN_COLLECTION:
    meta_data = collect_info()
    if SAVE_DATA:
        save_meta_data(meta_data, 'meta_data', APDB.literature_path)

else:
    meta_data = load_meta_data('meta_data', APDB.literature_path)

In [ ]:
# Check the meta data for the collection
meta_data

## Add & Check Terms

Now we can add these terms to LISC objects, ready for data collection. 

In [ ]:
# Initialize & add terms to the LISC object - concepts
counts_concepts = Counts()
counts_concepts.add_terms(concept_terms_dict, dim='A')
counts_concepts.add_terms(assoc_terms, dim='B')

In [ ]:
# Initialize & add terms to the LISC object - methods
counts_methods = Counts()
counts_methods.add_terms(method_terms_dict, dim='A')
counts_methods.add_terms(assoc_terms, dim='B')

In [ ]:
# Initialize & add terms to the LISC object - measures (no associations)
counts_measures = Counts1D()
counts_measures.add_terms(measure_terms_dict)

In [ ]:
# Check terms - concepts
counts_concepts.terms['A'].check_terms()

In [ ]:
# Check terms - methods
counts_methods.terms['A'].check_terms()

In [ ]:
# Check terms - measures
counts_measures.check_terms()

In [ ]:
# Check terms - associations
counts_concepts.terms['B'].check_terms()

## Collect Literature Data

Run literature collection. 

In [ ]:
# Run the data collection
if RUN_COLLECTION:
    counts_concepts.run_collection(db=db)
    counts_methods.run_collection(db=db)
    counts_measures.run_collection(db=db)
else:
    counts_concepts = load_object('counts_concepts.p', directory=APDB.literature_path)
    counts_methods = load_object('counts_methods.p', directory=APDB.literature_path)
    counts_measures = load_object('counts_measures.p', directory=APDB.literature_path)

## Check and Process Collected Data

Now we can examine the collected literature data.

In [ ]:
# Check how many papers were found for each term (concepts)
counts_concepts.check_counts()

In [ ]:
# Check how many papers were found for each term (methods)
counts_methods.check_counts()

In [ ]:
# Check how many papers were found for each term (measures)
counts_measures.check_counts()

In [ ]:
# Check the total number of papers across all terms
print('Total number of aperiodic papers (concepts): \t',
      sum(counts_concepts.terms['A'].counts))
print('Total number of aperiodic papers (methods): \t',
      sum(counts_methods.terms['A'].counts))
print('Total number of aperiodic papers (measures): \t',
      sum(counts_measures.counts))

In [ ]:
# Normalize the association results
counts_concepts.compute_score('normalize', dim='A')
counts_methods.compute_score('normalize', dim='A')

## Create Visualizations of Collected Data

### Concepts

In [ ]:
# Plot the associations of aperiodic measures in the literature
plot_matrix(counts_concepts.score, counts_concepts.terms['B'].labels,
            counts_concepts.terms['A'].labels, cmap='blue', square=True,
            xtickrotation=45, figsize=(8, 6), **fsaver('lit_counts_concepts'))

In [ ]:
# Plot the number of papers found per concept term
plot_vector(counts_concepts, cmap='blue', figsize=(1.5, 4), **fsaver('total_counts_concepts'))

### Methods

In [ ]:
# Plot the associations of aperiodic measures in the literature
plot_matrix(counts_methods.score, counts_methods.terms['B'].labels,
            counts_methods.terms['A'].labels, cmap='blue', square=True,
            xtickrotation=45, figsize=(8, 6), **fsaver('lit_counts_methods'))

In [ ]:
# Plot the number of papers found per methods measure
plot_vector(counts_methods, cmap='blue', figsize=(2, 5), **fsaver('total_counts_methods'))

### Measures

In [ ]:
# Plot the number of papers found per measure
with sns.plotting_context('notebook'):
    plot_vector(counts_measures, labels=counts_measures.labels, cmap='blue',
                transpose=True, xtickrotation=45, figsize=(15, 1.5),
                **fsaver('total_counts_measures'))

In [ ]:
# Save out the collected data
if SAVE_DATA:
    save_object(counts_concepts, 'counts_concepts', directory=APDB.literature_path)
    save_object(counts_methods, 'counts_methods', directory=APDB.literature_path)
    save_object(counts_measures, 'counts_measures', directory=APDB.literature_path)

## Examine Literature Terms over Time


In [ ]:
# Define range of years to collect data for
start_year, end_year = 1960, 2024
years = list(range(start_year, end_year + 1))

In [ ]:
# Collect count of the number of aperiodic related papers per year
if RUN_COLLECTION:
    
    # Setup object and run time collecting - concepts
    years_concepts = Counts1D()
    years_concepts.add_terms(concept_terms_dict)
    years_concept_results = collect_across_time(years_concepts, years)

    # Setup object and run time collecting - methods
    years_methods = Counts1D()
    years_methods.add_terms(method_terms_dict)
    years_method_results = collect_across_time(years_methods, years)

    if SAVE_DATA:
        save_time_results(years_concept_results, 'time_concepts', 'concepts', APDB.literature_path)
        save_time_results(years_method_results, 'time_methods', 'methods', APDB.literature_path)

else:
    years_concept_results = load_time_results('time_concepts', 'concepts', directory=APDB.literature_path)
    years_method_results = load_time_results('time_methods', 'methods', directory=APDB.literature_path)

In [ ]:
# Plot the results over time of the concept results
plot_results_across_years(years_concept_results, xlim=[1959.5, 2023.5], ylim=[0, 55],
                          figsize=(9, 4), **fsaver('years_concepts'))

In [ ]:
# Plot the results over time of the method results
plot_results_across_years(years_method_results, xlim=[1959.5, 2023.5], ylim=[0, 75],
                          figsize=(9, 4), **fsaver('years_methods'))